<a href="https://colab.research.google.com/github/robertoarturomc/ProgramacionConcurrente/blob/main/24_Introduccion_a_Spark_III.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Programación Concurrente
## 24. Introducción a Spark III



In [6]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit



In [2]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

Vamos a profundizar en algunos conceptos de PySpark. Para ello, continuaremos trabajando con la misma base de datos.

In [4]:
df = spark.read.csv('covidmex.csv', header=True)
df.show(5)

+-----------+-----------+-------------+-------------+--------------+------+-----+------------------+-------------------+------------------+--------------------+---------+------------------+
|ID_REGISTRO|ENTIDAD_RES|MUNICIPIO_RES|FECHA_INGRESO|FECHA_SINTOMAS|covidt|delta|               lat|               long|               alt|                 qry|dayofyear|       lengthofday|
+-----------+-----------+-------------+-------------+--------------+------+-----+------------------+-------------------+------------------+--------------------+---------+------------------+
|     z526b3|          9|           12|   2020-12-21|    2020-12-18|     1|    3| 19.20155345588235| -99.20180252450989|3008.9460784313724|Cve_Ent==9 & Cve_...|      356|11.768371962176587|
|     z3d1e2|          9|            5|   2020-04-22|    2020-04-20|     1|    2|         19.482945|         -99.113471|            2229.0|Cve_Ent==9 & Cve_...|      113|13.441805501598786|
|     z21f6f|          7|            9|   2020-04-

¿Cómo extraigo una parte de un String?



In [19]:
from pyspark.sql.functions import substring, lower, upper

In [15]:
df.withColumn('Subst', substring(col('ID_REGISTRO'), 1, 1)).show(5)

+-----------+-----------+-------------+-------------+--------------+------+-----+-----------------+------------------+------------------+--------------------+---------+------------------+-----+
|ID_REGISTRO|ENTIDAD_RES|MUNICIPIO_RES|FECHA_INGRESO|FECHA_SINTOMAS|covidt|delta|              lat|              long|               alt|                 qry|dayofyear|       lengthofday|Subst|
+-----------+-----------+-------------+-------------+--------------+------+-----+-----------------+------------------+------------------+--------------------+---------+------------------+-----+
|     z526b3|          9|           12|   2020-12-21|    2020-12-18|     1|    3|19.20155345588235|-99.20180252450989|3008.9460784313724|Cve_Ent==9 & Cve_...|      356|11.768371962176587|    z|
|     z3d1e2|          9|            5|   2020-04-22|    2020-04-20|     1|    2|        19.482945|        -99.113471|            2229.0|Cve_Ent==9 & Cve_...|      113|13.441805501598786|    z|
|     z21f6f|          7|     

¿Y si, además, quiero transformar en Mayúsculas/Minúsculas?

In [20]:
df.withColumn('Subst', substring(col('ID_REGISTRO'), 1, 1))\
  .withColumn('Minusc', lower(col('Subst')))\
  .withColumn('Mayusc', upper(col('Subst')))\
  .show(5)

+-----------+-----------+-------------+-------------+--------------+------+-----+------------------+-------------------+------------------+--------------------+---------+------------------+-----+------+------+
|ID_REGISTRO|ENTIDAD_RES|MUNICIPIO_RES|FECHA_INGRESO|FECHA_SINTOMAS|covidt|delta|               lat|               long|               alt|                 qry|dayofyear|       lengthofday|Subst|Minusc|Mayusc|
+-----------+-----------+-------------+-------------+--------------+------+-----+------------------+-------------------+------------------+--------------------+---------+------------------+-----+------+------+
|     z526b3|          9|           12|   2020-12-21|    2020-12-18|     1|    3| 19.20155345588235| -99.20180252450989|3008.9460784313724|Cve_Ent==9 & Cve_...|      356|11.768371962176587|    z|     z|     Z|
|     z3d1e2|          9|            5|   2020-04-22|    2020-04-20|     1|    2|         19.482945|         -99.113471|            2229.0|Cve_Ent==9 & Cve_...|

In [24]:
# Alternativamente
df.withColumn('Subst', substring(col('ID_REGISTRO'), 1, 1))\
  .select(lower(col('Subst')), upper(col('Subst')))\
  .show(5)

+------------+------------+
|lower(Subst)|upper(Subst)|
+------------+------------+
|           z|           Z|
|           z|           Z|
|           z|           Z|
|           z|           Z|
|           z|           Z|
+------------+------------+
only showing top 5 rows



¿Cómo calculo el mínimo/máximo?

In [25]:
from pyspark.sql.functions import min, max

In [27]:
df.select(min(col('FECHA_INGRESO')), max(col('FECHA_INGRESO')))\
  .show()

+------------------+------------------+
|min(FECHA_INGRESO)|max(FECHA_INGRESO)|
+------------------+------------------+
|        2020-02-27|        2022-02-08|
+------------------+------------------+



¿Cómo lo haría con `.withColumn()` ?


¿Y si quiero calcular qué día es, 14 días después de la fecha máxima de Ingreso?


In [34]:
from pyspark.sql.functions import date_add, date_sub

In [35]:
df.select(max(col('FECHA_INGRESO')).alias('MAX_ING'))\
  .select(date_add(col('MAX_ING'), 14))\
  .show()

+---------------------+
|date_add(MAX_ING, 14)|
+---------------------+
|           2022-02-22|
+---------------------+



Lo anterior, ¿se puede hacer en una sola línea?

Por cierto, trabajar con fechas tiene su chiste. Vamos a ver un par de casos interesantes.

In [36]:
from pyspark.sql.functions import to_date, to_timestamp

In [41]:
df.select(to_date(col('FECHA_INGRESO'), 'yyyy-MM-dd'), \
          to_timestamp(col('FECHA_INGRESO'), 'yyyy-MM-dd')).show(3)

+----------------------------------+---------------------------------------+
|to_date(FECHA_INGRESO, yyyy-MM-dd)|to_timestamp(FECHA_INGRESO, yyyy-MM-dd)|
+----------------------------------+---------------------------------------+
|                        2020-12-21|                    2020-12-21 00:00:00|
|                        2020-04-22|                    2020-04-22 00:00:00|
|                        2020-04-27|                    2020-04-27 00:00:00|
+----------------------------------+---------------------------------------+
only showing top 3 rows



In [44]:
df.select(col('FECHA_INGRESO'), \
          to_date(col('FECHA_INGRESO'), 'yyyy-MM-dd'), \
          to_timestamp(col('FECHA_INGRESO'), 'yyyy-MM-dd')).printSchema()

root
 |-- FECHA_INGRESO: string (nullable = true)
 |-- to_date(FECHA_INGRESO, yyyy-MM-dd): date (nullable = true)
 |-- to_timestamp(FECHA_INGRESO, yyyy-MM-dd): timestamp (nullable = true)



Y al igual que siempre, trabajar con tipos de datos es interesante. Aunque Pyspark puede "adivinar" el tipo de dato con el que estoy trabajando, no está de más el hacerlo de manera manual, para agregar una capa extra de seguridad.

In [46]:
df.select(col('covidt'), col('delta')).printSchema(3)

root
 |-- covidt: string (nullable = true)
 |-- delta: string (nullable = true)



In [45]:
df.select(col('covidt') + col('delta')).show(3)

+----------------+
|(covidt + delta)|
+----------------+
|             4.0|
|             3.0|
|             4.0|
+----------------+
only showing top 3 rows



In [47]:
df.select(col('covidt'), col('delta'), col('covidt') + col('delta')).printSchema(3)

root
 |-- covidt: string (nullable = true)
 |-- delta: string (nullable = true)
 |-- (covidt + delta): double (nullable = true)



¿Cómo cambio manualmente el tipo de Dato en Pyspark?

In [49]:
from pyspark.sql.types import StringType, IntegerType, DateType

In [ ]:
df.dtypes

In [53]:
df = df.withColumn('covidt_int', df['covidt'].cast(IntegerType()))
#df = df.withColumn('covidt_int', col('covidt').cast(IntegerType()))

df.dtypes

[('ID_REGISTRO', 'string'),
 ('ENTIDAD_RES', 'string'),
 ('MUNICIPIO_RES', 'string'),
 ('FECHA_INGRESO', 'string'),
 ('FECHA_SINTOMAS', 'string'),
 ('covidt', 'string'),
 ('delta', 'string'),
 ('lat', 'string'),
 ('long', 'string'),
 ('alt', 'string'),
 ('qry', 'string'),
 ('dayofyear', 'string'),
 ('lengthofday', 'string'),
 ('covidt_int', 'int')]

También funciona para Dates (no para Timestamps)

(Y nota cómo también puedo sobrescribir columnas)

In [54]:
df = df.withColumn('FECHA_INGRESO', df['FECHA_INGRESO'].cast(DateType()))

df.dtypes

[('ID_REGISTRO', 'string'),
 ('ENTIDAD_RES', 'string'),
 ('MUNICIPIO_RES', 'string'),
 ('FECHA_INGRESO', 'date'),
 ('FECHA_SINTOMAS', 'string'),
 ('covidt', 'string'),
 ('delta', 'string'),
 ('lat', 'string'),
 ('long', 'string'),
 ('alt', 'string'),
 ('qry', 'string'),
 ('dayofyear', 'string'),
 ('lengthofday', 'string'),
 ('covidt_int', 'int')]

Y funciona de regreso

In [55]:
df = df.withColumn('covidt_int_str', df['covidt_int'].cast(StringType()))

df.dtypes

[('ID_REGISTRO', 'string'),
 ('ENTIDAD_RES', 'string'),
 ('MUNICIPIO_RES', 'string'),
 ('FECHA_INGRESO', 'date'),
 ('FECHA_SINTOMAS', 'string'),
 ('covidt', 'string'),
 ('delta', 'string'),
 ('lat', 'string'),
 ('long', 'string'),
 ('alt', 'string'),
 ('qry', 'string'),
 ('dayofyear', 'string'),
 ('lengthofday', 'string'),
 ('covidt_int', 'int'),
 ('covidt_int_str', 'string')]

Recuérdenme, ¿cómo muestro sólo las columnas 'covidt', 'covidt_int' y 'covidt_int_str'?

Una comparativa entre Spark.SQL y Pyspark

In [79]:
df.createOrReplaceTempView('covid')

result = spark.sql('''SELECT
                        FECHA_INGRESO,
                        DATE(FECHA_INGRESO) AS FECHA_INGRESO_DAT,
                        FECHA_INGRESO_DAT + 14
                    FROM covid
                    ''')

result.show(5)

+-------------+-----------------+------------------------------------------------------+
|FECHA_INGRESO|FECHA_INGRESO_DAT|date_add(lateralAliasReference(FECHA_INGRESO_DAT), 14)|
+-------------+-----------------+------------------------------------------------------+
|   2020-12-21|       2020-12-21|                                            2021-01-04|
|   2020-04-22|       2020-04-22|                                            2020-05-06|
|   2020-04-27|       2020-04-27|                                            2020-05-11|
|   2020-09-06|       2020-09-06|                                            2020-09-20|
|   2020-07-10|       2020-07-10|                                            2020-07-24|
+-------------+-----------------+------------------------------------------------------+
only showing top 5 rows



In [71]:
# No se puede dentro del mismo .select() ... ¿O sí?
df.select(to_date(col('FECHA_INGRESO'), 'yyyy-MM-dd').alias('FECHA_INGRESO_DAT'),
          max(col('INGRESO_DATE'))).show(3)


AnalysisException: [MISSING_GROUP_BY] The query does not include a GROUP BY clause. Add GROUP BY or turn it into the window functions using OVER clauses.;
Aggregate [to_date(FECHA_INGRESO#766, Some(yyyy-MM-dd), Some(Etc/UTC), false) AS INGRESO_DATE#895, max(lateralAliasReference(INGRESO_DATE)) AS max(lateralAliasReference(INGRESO_DATE))#897]
+- Project [ID_REGISTRO#17, ENTIDAD_RES#18, MUNICIPIO_RES#19, FECHA_INGRESO#766, FECHA_SINTOMAS#21, covidt#22, delta#23, lat#24, long#25, alt#26, qry#27, dayofyear#28, lengthofday#29, covidt_int#751, cast(covidt_int#751 as string) AS covidt_int_str#781]
   +- Project [ID_REGISTRO#17, ENTIDAD_RES#18, MUNICIPIO_RES#19, cast(FECHA_INGRESO#20 as date) AS FECHA_INGRESO#766, FECHA_SINTOMAS#21, covidt#22, delta#23, lat#24, long#25, alt#26, qry#27, dayofyear#28, lengthofday#29, covidt_int#751]
      +- Project [ID_REGISTRO#17, ENTIDAD_RES#18, MUNICIPIO_RES#19, FECHA_INGRESO#20, FECHA_SINTOMAS#21, covidt#22, delta#23, lat#24, long#25, alt#26, qry#27, dayofy

In [73]:
df.select(to_date(col('FECHA_INGRESO'), 'yyyy-MM-dd').alias('FECHA_INGRESO_DAT'))\
  .select(max(col('FECHA_INGRESO_DAT')).alias('MAX_DATE'))\
  .select(date_add(col('MAX_DATE'), 14))\
  .show(3)


+----------------------+
|date_add(MAX_DATE, 14)|
+----------------------+
|            2022-02-22|
+----------------------+



In [82]:
# Mentí. Sí se puede "en una sola línea"
df.select(date_add(\
                   max(\
                       to_date(col('FECHA_INGRESO'), 'yyyy-MM-dd')),
                   14))\
  .show(3)


+-----------------------------------------------------+
|date_add(max(to_date(FECHA_INGRESO, yyyy-MM-dd)), 14)|
+-----------------------------------------------------+
|                                           2022-02-22|
+-----------------------------------------------------+

